# Manipulando Jobs do watsonx.ai através da API
Este notebook demonstra como usar a API do watsonx.ai para criar e executar jobs. Ele fornece um guia passo a passo sobre como autenticar na API, listar jobs, criar um novo job, criar uma execução de job e recuperar o status e os logs da execução do job.

#### Observação 1: Esse notebook foi planejado para ser executado dentro de um projeto do watsonx.ai. Grande parte dele até poderia ser executado localmente, mas algumas partes não funcionarão como planejado.

#### Observação 2: No repositório do GitHub desse notebook, foi disponibilizado para você um outro notebook auxiliar chamado Hello World Job. Ele será usado como exemplo para ser executado via API. Adicione-o també ao seu projeto.

## Importando Bibliotecas
Esta seção importa as bibliotecas necessárias para o notebook. Essas bibliotecas incluem o `IAM Authenticator` e a biblioteca `requests` para fazer solicitações HTTP.

In [2]:
from ibm_cloud_sdk_core import IAMTokenManager
from ibm_cloud_sdk_core.authenticators import IAMAuthenticator, BearerTokenAuthenticator
import requests
import os
import json

## Autenticação
Para autenticar na API vamos precisar de uma API Key da IBM Cloud. Abaixo estão os passos para obter uma.

#### Vá para o seu IBM Cloud Console e navegue até "Gerenciar" -> "Acesso".
<img title="Go to Manage -> Access" alt="Go to Manage -> Access" src=./images/apikey-1.png>

#### No menu à direita, navegue até API Keys. Crie uma nova API Key. Você pode dar a ela qualquer nome que desejar.
<img title="Ir para Gerenciar -> Acesso" alt="Ir para Gerenciar -> Acesso" src=./images/apikey-2.png>

#### Copie sua Chave de API e cole no seu Notebook no espaço indicado abaixo. 
<img title="Vá para Gerenciar -> Acesso" alt="Vá para Gerenciar -> Acesso" src=./images/apikey-3.png>

Esta seção autentica o usuário usando a usa API Key. Através do IAM Token Manager recuperamos um token de acesso que é usado para autenticar solicitações de API subsequentes.

In [ ]:
apikey = "SUA API KEY"
access_token = IAMTokenManager(
        apikey=apikey,
        url="https://iam.cloud.ibm.com/identity/token"
).get_token()

## Configurando a URL da API
Esta seção configura a URL base para a API do watsonx.ai. Esta URL é usada para construir os endpoints da API para solicitações subsequentes.

In [4]:
main_url = "https://api.dataplatform.cloud.ibm.com"

## Obtendo o ID de Projeto
Esta seção recupera o ID do seu projeto pelas variáveis de ambiente. Esse ID é uma variável obrigatória na URL da API. 

In [ ]:
project_id = os.getenv("PROJECT_ID")

## Configurando os cabeçalhos da API
Esta seção configura os cabeçalhos da API. Os cabeçalhos incluem o token de acesso.

In [6]:
headers = {
    'Authorization':'Bearer '+access_token,
    'Content-Type': 'application/json',
    'Accept': 'application/json'
}

## Listando Jobs
Esta seção lista todos os jobs no projeto especificado. Ela faz uma requisição GET para o endpoint `/v2/jobs` e retorna uma resposta JSON contendo a lista de jobs.

In [7]:
requests.get(main_url+"/v2/jobs?project_id="+project_id, headers=headers).json()

{'total_rows': 0, 'results': []}

## Listando Notebooks
Esta seção lista todos os notebooks no projeto especificado. Ela faz uma requisição para listar os assets do tipo 'notebook' usando a biblioteca `wslib`. Para usar a biblioteca, precisamos fazer alguns passos antes.

#### Vá para a aba Gerenciar no seu projeto no watsonx.ai. No menu à esquerda, selecione Controle de Acesso.
<img title="Ir para Gerenciar -> Acesso" alt="Ir para Gerenciar -> Acesso" src=./images/project-token-1.png>

#### Crie um novo Token com a função de Editor. Você pode dar qualquer nome a ele.
<img title="Crie um novo Access Token com a função de Editor. Você pode dar qualquer nome a ele." alt="Vá para Gerenciar -> Acesso" src=./images/project-token-2.png>

#### Veja o token recém-criado na lista.
<img title="Ir para Gerenciar -> Acesso" alt="Ir para Gerenciar -> Acesso" src=./images/project-token-3.png>

#### Adicione seu token e a biblioteca de Gerenciamento de Projetos (wslib) ao seu Notebook. O botão indicado na imagem adicionará uma nova célula com o token e o código para usá-lo.
<img title="Ir para Gerenciar -> Acesso" alt="Ir para Gerenciar -> Acesso" src=./images/project-token-4.png>

Depois de executar a célula que vai ser adicionada automaticamente ao notebook (não mostrada aqui pois contém credenciais), podemos usar a `wslib` para listar os notebooks.

In [ ]:
wslib.assets.list_assets(asset_type='notebook')

#### Aqui você deveria ver o Notebook demonstrativo Hello World Job mencionado no início. Se você não o adicionou ainda ao seu projeto, faça isso agora e execute a célula novamente. Armazene o ID do notebook na variável abaixo.

In [ ]:
selected_notebook_asset_id = 'Hello World Job ID'

## Criando um Job
Esta seção cria um novo job para executar o notebook especificado, passando como input uma variável de ambiente. Ela faz uma solicitação POST para o endpoint `/v2/jobs` com um JSON contendo a configuração do job.

In [ ]:
data_job = {
    'job': {
        'name': 'Job API',
        'configuration': {
            'env_variables': ['TEST_VAR=%22Hello%22'],
            'run_results': 'log_and_new_notebook',
            'env_type': 'notebook'
        },
        'asset_ref': selected_notebook_asset_id,
    }
}

job_response = requests.post(main_url+"/v2/jobs?project_id="+project_id, headers=headers, data=json.dumps(data_job)).json()
job_response

#### Salve o seu job id para nossas próximas etapas

In [ ]:
job_id = job_response['metadata']['asset_id']

## Salvando uma Versão do Notebook
Esta seção cria uma nova versão (checkpoint) para o seu Notebook. Um Job precisa executar uma versão do notebook (por padrão será a mais recente, mas também pode ser uma específica). Ela faz uma solicitação POST para `/v2/notebooks/{notebook_asset_id}/versions`. Salvar uma versão do notebook também pode ser feito através da interface.

In [ ]:
version_response = requests.post(main_url+f"/v2/notebooks/{selected_notebook_asset_id}/versions?project_id={project_id}", headers=headers, data=json.dumps({})).json()
version_response

## Criando uma Execução de Job
Esta seção cria uma nova execução para o job especificado. Ela faz uma solicitação POST para o endpoint `/v2/jobs/{job_id}/runs` com um JSON contendo a configuração da execução do job.

In [ ]:
data_run_1 = {
    'job_run': {
        'name': 'Job Run API',
        # 'configuration': {
        #     'env_variables': ['TEST_VAR=%22Hello2%22'],
        # },
    }
}

run_response_1 = requests.post(main_url+f"/v2/jobs/{job_id}/runs?project_id={project_id}", headers=headers, data=json.dumps(data_run_1)).json()
run_response_1

#### Salve o ID da execução do seu trabalho para nossas próximas etapas

In [ ]:
job_run_id = run_response_1['metadata']['asset_id']

## Obtendo o Status da Execução do Job
Esta seção recupera o status da execução do job. Ela faz uma solicitação GET para o endpoint `/v2/jobs/{job_id}/runs/{run_id}` e retorna uma resposta JSON contendo o status da execução do job.

In [ ]:
run_status = requests.get(main_url+f"/v2/jobs/{job_id}/runs/{job_run_id}?project_id={project_id}", headers=headers).json()
run_status

## Obtendo os Logs de Execução do Job
Esta seção recupera os logs para a execução do job. Ele faz uma solicitação GET para o endpoint `/v2/jobs/{job_id}/runs/{run_id}/logs` e retorna uma resposta JSON contendo os logs de execução do job.

In [30]:
requests.get(main_url+f"/v2/jobs/{job_id}/runs/{job_run_id}/logs?project_id={project_id}", headers=headers).json()

{'results': ['', 'Cell 3:', '', '', 'Cell 5:', 'TEST_VAR: "Hello"', ''],
 'total_count': 7}

## Execução de Job com variáveis de ambiente
Esta seção cria uma nova execução de job com variáveis de ambiente. Isso sobrescreverá as variáveis que já existem na definição do job.

In [ ]:
data_run_2 = {
    'job_run': {
        'name': 'Job Run API w/ Env Var',
        'configuration': {
            'env_variables': ['TEST_VAR=%22Hello2%22'],
        },
    }
}

run_response_2 = requests.post(main_url+f"/v2/jobs/{job_id}/runs?project_id={project_id}", headers=headers, data=json.dumps(data_run_2)).json()
run_response_2

In [33]:
job_run_id = run_response_2['metadata']['asset_id']

In [34]:
requests.get(main_url+f"/v2/jobs/{job_id}/runs/{job_run_id}/logs?project_id={project_id}", headers=headers).json()

{'results': ['', 'Cell 3:', '', '', 'Cell 5:', 'TEST_VAR: "Hello2"', ''],
 'total_count': 7}